In [2]:
import pandas as pd

In [24]:
df_raw = pd.read_excel('data/Teeltplan 2024.xlsx')

/Users/svenvanderburg/miniforge3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [25]:
df_raw.head()

,Unnamed: 0,Gewas,Ras,Aantal vakken,Bed\nnummer,Z / PL / JONG,Combinatieteelt,Afstand tussen rij,Afstand in rij,Planten per vak,...,Plantdatum,Verwachte oogst (week),Startdatum oogst,Eindweek\noogst,Einddatum oogst,Dagen tot volgroeiing,Dagen de tuin,Dagen in opkweek,Leverancier,Opmerkingen
0,x,Aardbei,?,2.0,NaN,NaN,NaN,15.0,15.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,x,Wintertarwe,Porthus,4.0,NaN,Z,NaN,NaN,NaN,NaN,...,NaN,28.0,NaN,28.0,NaN,NaN,NaN,NaN,NaN,NaN
3,x,Oerrogge,SECALE multicaule 'Waldstauden',0.0,NaN,Z,NaN,NaN,NaN,NaN,...,NaN,28.0,NaN,28.0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df_raw.columns

Index(['Unnamed: 0', 'Gewas', 'Ras', 'Aantal vakken', 'Bed\nnummer',
       'Z / PL / JONG', 'Combinatieteelt', 'Afstand tussen rij',
       'Afstand in rij', 'Planten per vak', 'Planten nodig', 'Zaaien (week)',
       'Zaaidatum', 'Planten (week)', 'Plantdatum', 'Verwachte oogst (week)',
       'Startdatum oogst', 'Eindweek\noogst', 'Einddatum oogst',
       'Dagen tot volgroeiing', 'Dagen de tuin', 'Dagen in opkweek',
       'Leverancier', 'Opmerkingen'],
      dtype='object')

In [27]:
def get_unique_bed_numbers(df):
    cleaned = df['Bed\nnummer'].str.replace(" ","").str.split(',').dropna()
    return {e for l in cleaned for e in l}

data = list()
mapper = {
    'zaaien': 'Zaaien (week)',
    'planten': 'Planten (week)'} 
for week in range(2, 53):
    for activity in mapper.keys():
        selection = df_raw[df_raw[mapper[activity]] == week]
        for gewas in set(selection['Gewas']):
            this_rows = selection[selection['Gewas'] == gewas]
            # bed_numbers = get_unique_bed_numbers(this_rows)
            # if bed_numbers and next(iter(bed_numbers)) == '?':
            #     print(f'skipping {gewas} becaue of no bednumbers')
            #     continue
            if len(this_rows['Z / PL / JONG'].unique()) > 2:
                raise ValueError(f'Multiple methods for {gewas}')
            method = this_rows['Z / PL / JONG'].unique()[0]
            if activity in ['zaaien', 'planten'] and method == 'DZ':
                to_do = 'direct zaaien'
            elif activity == 'zaaien' and method == 'TP':
                to_do = 'voorzaaien'
                bed_numbers = []
            else:
                to_do = activity

                
            data.append({'Week': week,
                         'Wat doen?': f'{gewas} {to_do}'})
in_time_df = pd.DataFrame(data)

In [28]:
in_time_df.head()

,Week,Wat doen?
0,3,Eikenbladsla (kas vroeg) zaaien
1,5,Chinese bieslook zaaien
2,6,Chilipeper zaaien
3,6,Rankspinazie zaaien
4,7,Eikenbladsla (kas vroeg) planten


In [29]:
to_add = [
    {'Week': 13, 'Wat doen?': 'Roos, olijfboom snoeien'},
    {'Week': 13, 'Wat doen?': 'Vlinderstruik, braam snoeien'},
    {'Week': 10, 'Wat doen?': 'Rabarber scheuren'},
    {'Week': 14, 'Wat doen?': 'Lavendel snoeien'},
    {'Week': 6, 'Wat doen?': 'Winterjasmijn snoeien'}
]
in_time_df = in_time_df.append(to_add)
in_time_df = in_time_df.sort_values(by='Week')

/var/folders/ff/lzvrl_f53bl3fwlbvnsw4f840000gn/T/ipykernel_59419/2058819248.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  in_time_df = in_time_df.append(to_add)


In [30]:
in_time_df.to_excel('data/planning-per-week.xlsx', index=False)